<H3> VALUE AT RISK: A Comparison of Methods</H3>

<h3>Value at Risk </h3>
Value at Risk (VaR) is a widely used risk management metric that quantifies the potential loss in the value of a portfolio of financial instruments over a specific time horizon and with a given level of confidence. In other words, VaR provides an estimate of the maximum amount of loss that a portfolio could experience within a certain period at a certain probability level.

This project will compare three methods to calculate this metric - Monte Carlo Simulation, Parametric Methods, Historical.

<h3>Data Collection</h3>

1. Firstly, I will import libraries needed for this project.

In [7]:
# Import libraries
import pandas as pd              # Data management.
import numpy as np               # Linear algebra.
from scipy.stats import norm     # Normal distribution.
import matplotlib.pyplot as plt  # Plotting.
%matplotlib inline
import yfinance as yf            # Yahoo Finance API.
import datetime as dt            # Working with timestaps

2. Secondly, I will initiate variables that are going to be used in this project.<br>I will create an equally weighted portfolio of five ETFs with total value of $1000.<br>In this project, I will work with the rather standard value of 0.05 for the confidence level - the first component of Value at Risk.

In [15]:
portfolio       = ["AAPL", "MSFT", "AMZN", "TSLA", "JPM"]       # Create a list of stock tickers.
weights         = np.array([1/len(portfolio)]*len(portfolio))   # Create an array of weights (all equal to the reciprocal of the number of stocks in the portfolio).
portfolio_value = 1000                                          # Create a variable storing total value of the portfolio.
confidence      = 0.05                                          # Create a variable storing the confidence interval.
years_back      = 5                                             # Create a variable storing the number of years we want to get data for.
days            = 5                                             # Create a variable storing the number of days in the future we want to estimate risk for.

3. Now, I will use the YFinance API to collect stock prices for each security in the portfolio.<br>I will use adjusted stock prices, which incorporate any dilutive operations, such as splits or dividends. Thanks to that, the stock price can be compared throughout years.

In [11]:
df = yf.download(portfolio, start = dt.datetime.now() - dt.timedelta(days = 365*int(years_back)), end = dt.datetime.now(), progress = False)["Adj Close"]

4. Compute certain statistics.

In [12]:
# Convert stock prices to daily returns.
returns         = df.pct_change()
# Compute the mean of daily returns for each stock.
mean_returns    = returns.mean()
# Compute portfolio average return using the previously established weights.
portfolio_mean  = mean_returns.dot(weights)
# Compute average return on investment using the previously established portfolio value.
investment_mean = (1 + portfolio_mean) * portfolio_value
# Compute the covariance matrix of the portfolio - a matrix representing covariance between each pair of stocks in the portfolio.
cov_matrix      = returns.cov() 
# Compute the standard deviation of the portfolio. 
# I will use the following formula for to calculate the variance: transposed matrix of weights dotted with covariance matrix and dotted with the matrix of weights. 
portfolio_sd    = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
# Compute the standard deviation of returns.
investment_sd   = portfolio_value * portfolio_sd

5. Create a function that will generate random scenarios of gains or losses based on randomly generated Z-scores.

In [16]:
def scenario_generator(value, exp_rets, st_dev, z, days):
    # Compute expected returns based on previous results.
    returns   = value * exp_rets * days
    # Compute random gain/loss using the randomly generated z_score.
    loss_gain = value * st_dev * z * np.sqrt(days) 
    # Return.
    return returns + loss_gain

6. Run the Monte Carlo simulation. I will generate 10000 random scenarios based on the standard normal distribution, meaining that values appearing with the highest probability will be those from the middle section of the bell-curve.

In [17]:
# Set up the number of simulations.
simulations = 10000
# Initiate a list to store outcomes in.
results     = []

# Loop to create a 10000 possible scenarios.
for i in range(simulations):
    z = np.random.normal(0,1)
    results.append(scenario_generator(portfolio_value, portfolio_mean, portfolio_sd, z, days))

7. Compute the Value at Risk statistic using the previously established confidence level.

In [37]:
# Compute the final result.
VAR = -np.percentile(results, 100*(confidence))
# Report the result.
print(f"For {days} days long time interval and confidence level of {confidence}, Value at Risk of the portfolio is ${VAR.round(2)} (${portfolio_value} was invested).")

For 5 days long time interval and confidence level of 0.05, Value at Risk of the portfolio is $53.71 ($1000 was invested).
